In [9]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#Enter details of your Azure Machine Learning workspace
subscription_id = '80c77c76-74ba-4c8c-8229-4c3b2957990c'
resource_group = 'sonata-test-rg'
workspace_name = 'sonata-test-ws'

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace_name)

In [10]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

In [11]:
from azure.ai.ml import command, Input

# define the command
command_job = command(
    code="./",
    # command="python gpt2_trainer.py --cnn_dailymail ${{inputs.cnn_dailymail}}",
    command="python gpt2_trainer.py",
    environment="gpt2-venv:7",
    # inputs={
    #     "cnn_dailymail": Input(
    #         type="uri_file",
    #         path="https://datasets-server.huggingface.co/rows?dataset=cnn_dailymail&config=3.0.0&split=validation&offset=0&limit=5",
    #     )
    # },
    compute="cpu-cluster",
)

In [12]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Uploading Job (0.04 MBs): 100%|██████████| 38820/38820 [00:00<00:00, 422052.74it/s]




'https://ml.azure.com/runs/clever_bridge_3lnly286nh?wsid=/subscriptions/80c77c76-74ba-4c8c-8229-4c3b2957990c/resourcegroups/sonata-test-rg/workspaces/sonata-test-ws&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'